In [ ]:
# default_exp problem_types.seq2seq_text
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
# test setup
import tensorflow as tf
import numpy as np
from m3tl.test_base import TestBase
from m3tl.input_fn import train_eval_input_fn
from m3tl.test_base import test_top_layer
test_base = TestBase()
params = test_base.params

hidden_dim = params.bert_config.hidden_size

train_dataset = train_eval_input_fn(params=params)
one_batch = next(train_dataset.as_numpy_iterator())


# Seq2seq text generation(seq2seq_text)

This module includes neccessary part to register Seq2seq text generation problem type.

THIS IS NOT IMPLEMENTED YET!!!

## Imports and utils


In [ ]:
# export
import pickle
from typing import Dict, List, Tuple

import numpy as np
import tensorflow as tf
from m3tl.base_params import BaseParams
from m3tl.utils import (LabelEncoder, get_label_encoder_save_path,
                        load_transformer_tokenizer, need_make_label_encoder)


## Top Layer

In [ ]:
# export

class Seq2Seq(tf.keras.Model):
    def __init__(self, params: BaseParams, problem_name: str, input_embeddings: tf.keras.layers.Layer):
        super(Seq2Seq, self).__init__(name=problem_name)
        # self.params = params
        # self.problem_name = problem_name
        # # if self.params.init_weight_from_huggingface:
        # #     self.decoder = load_transformer_model(
        # #         self.params.transformer_decoder_model_name,
        # #         self.params.transformer_decoder_model_loading)
        # # else:
        # #     self.decoder = load_transformer_model(
        # #         self.params.bert_decoder_config, self.params.transformer_decoder_model_loading)

        # # TODO: better implementation
        # logging.warning(
        #     'Seq2Seq model is not well supported yet. Bugs are expected.')
        # config = self.params.bert_decoder_config
        # # some hacky approach to share embeddings from encoder to decoder
        # word_embedding_weight = input_embeddings.word_embeddings
        # self.vocab_size = word_embedding_weight.shape[0]
        # self.share_embedding_layer = TFSharedEmbeddings(
        #     vocab_size=word_embedding_weight.shape[0], hidden_size=word_embedding_weight.shape[1])
        # self.share_embedding_layer.build([1])
        # self.share_embedding_layer.weight = word_embedding_weight
        # # self.decoder = TFBartDecoder(
        # #     config=config, embed_tokens=self.share_embedding_layer)
        # self.decoder = TFBartDecoderForConditionalGeneration(
        #     config=config, embedding_layer=self.share_embedding_layer)
        # self.decoder.set_bos_id(self.params.bos_id)
        # self.decoder.set_eos_id(self.params.eos_id)

        # self.metric_fn = tf.keras.metrics.SparseCategoricalAccuracy(
        #     name='{}_acc'.format(self.problem_name))
        raise NotImplementedError

    def _seq2seq_label_shift_right(self, labels: tf.Tensor, eos_id: int) -> tf.Tensor:
        batch_eos_ids = tf.fill([tf.shape(labels)[0], 1], eos_id)
        batch_eos_ids = tf.cast(batch_eos_ids, dtype=tf.int64)
        decoder_lable = labels[:, 1:]
        decoder_lable = tf.concat([decoder_lable, batch_eos_ids], axis=1)
        return decoder_lable

    def call(self,
             inputs: Tuple[Dict[str, Dict[str, tf.Tensor]], Dict[str, Dict[str, tf.Tensor]]],
             mode: str):
        features, hidden_features = inputs
        encoder_mask = features['model_input_mask']

        if mode == tf.estimator.ModeKeys.PREDICT:
            input_ids = None
            decoder_padding_mask = None
        else:
            input_ids = features['%s_label_ids' % self.problem_name]
            decoder_padding_mask = features['{}_mask'.format(
                self.problem_name)]

        if mode == tf.estimator.ModeKeys.PREDICT:
            return self.decoder.generate(eos_token_id=self.params.eos_id, encoder_hidden_states=hidden_features['seq'])
        else:
            decoder_output = self.decoder(input_ids=input_ids,
                                          encoder_hidden_states=hidden_features['seq'],
                                          encoder_padding_mask=encoder_mask,
                                          decoder_padding_mask=decoder_padding_mask,
                                          decode_max_length=self.params.decode_max_seq_len,
                                          mode=mode)
            loss = decoder_output.loss
            logits = decoder_output.logits
            self.add_loss(loss)
            decoder_label = self._seq2seq_label_shift_right(
                features['%s_label_ids' % self.problem_name], eos_id=self.params.eos_id)
            acc = self.metric_fn(decoder_label, logits)
            self.add_metric(acc)
            return logits

## Get or make label encoder function


In [ ]:
# export
def seq2seq_text_get_or_make_label_encoder_fn(params: BaseParams, problem: str, mode: str, label_list: List[str], *args, **kwargs) -> LabelEncoder:

    le_path = get_label_encoder_save_path(params=params, problem=problem)
    if need_make_label_encoder(mode=mode, le_path=le_path, overwrite=kwargs['overwrite']):
        # fit and save label encoder
        label_encoder = load_transformer_tokenizer(
            params.transformer_decoder_tokenizer_name, params.transformer_decoder_tokenizer_loading)
        pickle.dump(label_encoder, open(le_path, 'wb'))
        params.set_problem_info(problem=problem, info_name='num_classes', info=len(label_encoder.encode_dict))
    else:
        label_encoder = pickle.load(open(le_path, 'rb'))

    return label_encoder

## Label handing function

In [ ]:
# export
def pad_wrapper(inp, target_len=90):
    if len(inp) >= target_len:
        return inp[:target_len]
    else:
        return inp + [0]*(target_len - len(inp))

def seq2seq_text_label_handling_fn(target, label_encoder=None, tokenizer=None, decoding_length=None, *args, **kwargs):
    target = [label_encoder.bos_token] + \
        target + [label_encoder.eos_token]
    label_dict = label_encoder(
        target, add_special_tokens=False, is_split_into_words=True)
    label_id = label_dict['input_ids']
    label_mask = label_dict['attention_mask']
    label_id = pad_wrapper(label_id, decoding_length)
    label_mask = pad_wrapper(label_mask, decoding_length)
    return label_id, label_mask

